<a href="https://www.kaggle.com/code/goldens/don-t-let-the-clients-quit?scriptVersionId=143995911" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Churn (when clients leave)
## The problem: Customers are quiting the telecom service


The goal in this notebook is to explore the dataset and create a model to predict and understand more about the churn rate.
The dataset will be split: some new clients will be separeted to build a new dataset with the unchurned clients. This new dataset is for another part of this project.

# 0- Libraries import, file reading and general exploration

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
import ipyvuetify as v
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.utils import resample
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report


In [ ]:
df= pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum().sum()

In [ ]:
df_types= df.dtypes
row1= df.iloc[1]
row2= df.iloc[2]

explo_df = pd.DataFrame({'Types': df_types,'row1': row1, 'row2': row2})
explo_df

### Some changes needed: investigate SeniorCitizen and TotalCharges

In [ ]:
df['SeniorCitizen'] = df['SeniorCitizen'].astype(str)
df['SeniorCitizen'].dtype

In [ ]:
#df['TotalCharges'] = pd.to_numeric(df['TotalCharges']) # it will show that we have cells = " "
# as empty values on Total Charges don't make sense, they will be dropped
df = df.loc[df["TotalCharges"] != " "]
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

In [ ]:
# split pretending I don't have the results for new clients with less then 1 month
df_new = df.loc[(df["tenure"] <= 1) | (df['Churn']== "No")]

df = df.loc[df["tenure"] > 1]

In [ ]:
df_new["Churn"].value_counts()

# 1- Extensive EDA
Pandas profiling and Sweetviz are good option here but I wanted explore more the relationship between each independent variable x response variable.  
I am using Seaborn library here. Seaborn is a python library for statistical visualizations, built on top of matplotlib. Despite seaborn uses matplotlib behind the scenes, sometimes it's necessary to use matplotlib directly to do some customizations. (Source: https://seaborn.pydata.org/introduction.html)

Small review about visualization plots:

Distribution plots:

* Countplot: shows how many observations in each category.  
* Histogram: shows how many observations fall into an interval of continuous data.
* Kernel Density Estimate (KDE): shows the distribution of values with a continuous curve. It is analogous to a histogram. KDE uses a continuous probability density curve.
* Empirical Cumulative Distribution Function: maps values to its percentile rank and shows the percentage of values that are equal to or lower than this value.
* Stripplot: shows the distribution of values by category.
* Swarmplot: shows the distribution of values by category with ajusted points to avoid overlapping.
* Boxplot: shows the distribution highlighting quartiles and outliers.
* Violinplot: shows the distribution highlighting quartiles and outliers + kernel density plots on side.

Relationship plots:

* Barplot: shows, with bars, estimates of central tendency + confidence intervals (relationship between numeric and categoric).
* Scatterplot: shows the relationship between 2 numerical variables.
* Bubleplot: able to show the relationship among 3 numerical variables.
* Regression: helps visualizing a linear relationship.

## Churn, the response variable

In [ ]:
print(df['Churn'].apply(type).value_counts())
ax0=df['Churn'].value_counts(normalize=True).mul(100).round(1).plot(kind='bar',color=["#c2c0c0", "#c2c0c0"], title="Churn distribution",figsize=(4,3))
ax0.bar_label(ax0.containers[0],label_type="center",fmt='%.f%%')
plt.show()

<div class="alert alert-block alert-info">
Imbalanced dataset!
</div>

In [ ]:
##################################### function for exploration: plots + check if object type has only str

def explore_variable (variable):
    print(variable)
    if df[variable].dtype == "object":
        print(df[variable].apply(type).value_counts())
        ############ dfs
        df1=df[variable].value_counts(normalize=True).mul(100).round(1)
        df2=pd.crosstab(df[variable],df['Churn'], normalize='index').mul(100).round(1)
        ############ plots
        fig,axes = plt.subplots(1, 2, figsize=(8,3))
        ax1=df1.plot(kind='bar', color=["#c2c0c0", "#c2c0c0"], title=f"{variable} distribution", ax=axes[0]) # here color by bar
        ax1.bar_label(ax1.containers[0],label_type="center",fmt='%.f%%') 
        ax2=df2.plot(kind='bar', stacked=True, color=["#1b7df5", "#f51d82"],title=f"Churn distribution by {variable}", ax=axes[1]) # here color by subgroup
        ax2.legend(bbox_to_anchor=(1.0, 1.0))
        for c in ax2.containers:
            ax2.bar_label(c, label_type="center",fmt='%.f%%')

        plt.show()

    else:
        fig, axes = plt.subplots(1, 3, figsize=(11,3))
        sns.histplot(data=df, x=variable, kde=True, color="#c2c0c0", ax=axes[0]).set_title(f"{variable} distribution - histogram")
        sns.ecdfplot(data=df, x=variable, hue= 'Churn', palette=["#1b7df5", "#f51d82"], ax=axes[1]).set_title(f"{variable} distribution - ecdf plots")
        sns.violinplot(data=df, x='Churn', y=variable, palette=["#1b7df5", "#f51d82"], ax=axes[2]).set_title(f"{variable} distribution by Churn")
        plt.show()

In [ ]:
######################## exploration
explore_variable(list(df.columns)[1])

<div class="alert alert-block alert-info">
The gender is balanced and churn distribution for males and females are similar.
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[2])

<div class="alert alert-block alert-info">
16% of customers are Senior Citizen and they present a higher percentage of churn in comparison to the other group.
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[3])

<div class="alert alert-block alert-info">
52% of customers have no partner and this group have a higher percentage of churn in comparison to the other group.
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[4])

<div class="alert alert-block alert-info">
70% of clients have no dependents and these customers have a higher percentage of churn in comparison to the other group.
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[5])

<div class="alert alert-block alert-info">
Tenure represents the number of months the customer has stayed with the company.  
From the ECDF plots we already can see the difference between tenure distribution in groups "Churn" and "No Churn". 
Also, from the violin plots, at the inner boxplots, we can see that the median point of "No Churn" lies outside the "Yes Churn" box - so probably there is a significant difference between these distributions. Churn group has a higher frequence of lower values of tenure.
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[6])

<div class="alert alert-block alert-info">
90% of customers have a phone service. The Churn distribution on these two groups is similar.
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[7])

<div class="alert alert-block alert-info">
Churn distribution on these groups is also similar.
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[8])

<div class="alert alert-block alert-info">
The Fiber optic is the internet provider more frequent and the one with the higher proportion of Churn in this comparison
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[9])

<div class="alert alert-block alert-info">
Clients with no online security are 50% and they also have a higher proportion of Churn in this comparison
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[10])

<div class="alert alert-block alert-info">
Clients with no online backup are 44% and they also have a higher proportion of Churn in this comparison
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[11])

<div class="alert alert-block alert-info">
Similar behavior to Online Backup.
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[12])

<div class="alert alert-block alert-info">
Similar behavior to Online Backup.
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[13])

<div class="alert alert-block alert-info">
Clients with and without Streaming TV are almost balanced and they have close proportion of Churn (higher than group with no internet service)
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[14])

<div class="alert alert-block alert-info">
Similar behavior to Streaming TV
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[15])

<div class="alert alert-block alert-info">
The contract Month-to-Month is the most common and it has a higher proportion of Churn in comparioson to the other contracts
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[16])

<div class="alert alert-block alert-info">
59% of clients have paperless billing, a group that has higher proportion of Churn in comparison to those without Paperless billing
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[17])

<div class="alert alert-block alert-info">
The payment with Eletronic check has a higher proportion of Churn in comparison to other payment methods.
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[18])

<div class="alert alert-block alert-info">
There is a higher frequence of low monthly charges in the group "No Churn" and a slightly higher frequency on higher monthly charges at group "Churn".
</div>

In [ ]:
######################## exploration
explore_variable(list(df.columns)[19])

<div class="alert alert-block alert-info">
There is a higher frequency on lower total charges at group "Churn".
</div>

<div class="alert alert-block alert-success">
With this first exploration we may have an idea of the possible impact of some variables on churn rate: "no partners", "no dependents", Fiber Optic service, the lack of additional services and the contract month-to-month may be related to churn.
At the Churn group, the higher frequence of lower tenure and lower total charges, may indicate a dissatisfaction rounding new clients.
The "Churn Group" has a slightly higher frequency on higher monthly charges - was the cost/benefit relationship good for these client?
Paperless Billing and Eletronick Check payment method may have some relationship with Churn or another explanatory variable related to Churn.
</div>

### Possible Outliers
An outlier is a number numerically distant from the other numbers.  
One of the ways of identify outliers is using the boxplot: the box goes from the first to the third quartile (IQR distance) and, the "whiskers" go from these quartiles to the "minimum" and "maximum" value (excludind outliers).
Usually is defined as outlier the observation that is 1.5 IQR less the first quartile or 1.5 IQR greater then third quartile - they are the dots outside the range of the whiskers.

There are other methods to check for outliers but here I will start with this concept to give an extra attention to these samples deeply. Outliers may be dropped, transformed (maybe using np.log) or even another feature can be created: a boolean indicating if it is an outlier or not.




In [ ]:
list_of_numerics=df.select_dtypes(include=['float','int']).columns

possible_out = df.apply(lambda x: sum(
    (x < (x.quantile(0.25) - 1.5 * (x.quantile(0.75)- x.quantile(0.25))))|
    (x > (x.quantile(0.75) + 1.5 * (x.quantile(0.75)- x.quantile(0.25))))
) if x.name in list_of_numerics else '')

possible_out = pd.DataFrame({'NPO': possible_out})
possible_out

# 2. Data Preparation

With no null values and no outliers to remove, the focus will be:  
* transform categorical types (one hot encoding)
* balance the dataset

### 2.1 One Hot Encoding
one hot encoding is a transformation from categorical to numerical - that will avoid orders  
one hot encoding creates a new dummy feature for each unique value and binary values which will indicate presence or not

In [ ]:
df['Churn'].value_counts()

In [ ]:
# label encoding first so label will be numeric
from sklearn import preprocessing
enc= preprocessing.LabelEncoder()
df['Churn']= enc.fit_transform(df['Churn'])

In [ ]:
df['Churn'].value_counts()

In [ ]:
df2=pd.get_dummies(df.drop("customerID", axis=1), drop_first=True)# drop_first=True to reduce multicolinearity

### 2.2 Class Imbalance

So let's work on imbalance.There is a small sample with the positive class (26% of Churn).  
The main problem of work with imbalanced classes is that accuracy evaluation on test set won't be reliable. Why? The model can predict all the samples as the majority class and present a high accuracy.  
Some algo implementations in sklearn have parameters to work on weightning but here I am working on resampling the training dataset (otherwise the algorithm will learn a model that optimizes the prediction of the most present class) but testing with an imbalanced dataset to imitate real life. So, it can be necessary to check other evaluation metrics like precision (specificity) and recall (sensitivity).

   
**Precision** is about how good the model is to predict correctly as positive (get right, credibility) TP/TP+FP  
**Recall**   is about how good the model is to predict correctly the positives (reach them) TP/FN+TP   

I have used F1 on cross validation. F1-score is a combination of precision and recall = 2.(Precision x Recall/Precision + Recall)


In [ ]:
# separate train and test (after get dummy)
X= df2.drop("Churn", axis=1)
y= df2['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)#stratify=y for classes with same proportion

In [ ]:
# upsampling just for record - use this traindataset upsampled won't make a big difference here

train = pd.concat([X_train, y_train], axis=1)

minority_class = train[train["Churn"] == 1]
majority_class = train[train["Churn"] == 0]


upsampled_minority = resample(minority_class,
                              replace=True,
                              n_samples=len(majority_class),
                              random_state=42)

trainb = pd.concat([majority_class, upsampled_minority])
trainb = trainb.reset_index(drop=True)
#trainb['Churn'].value_counts()

X_trainb=trainb.drop('Churn', axis=1)
y_trainb=trainb['Churn']

# 3- Classification with Sklearn: Logistic Regression

Logistic regression is a type of generalized linear model, and can be used for classification.  
Logistic regression is close to Linear Regression.  

While linear regression works to fit a line to the data using least squares method , the logistic regression works to fit a s-shaped curve to the data using Maximum Likelihood Estimation.  
Least squares find the line that minimizes the sum of squared residuos and  Maximun Likelihood find the optimal values of mean and standard deviation for a distribution, given the data.


In [ ]:
pipe = make_pipeline(StandardScaler(), LogisticRegression()) # pipeline enables to chain transformers and estimator
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred))

# 4- Classification with Sklearn: Random Forest

Before Random Forest, let's remind something about Decision Trees. Roughly speaking, Decicion Trees split data making questions about feature values at each node. During this split there is the goal of maximizing the information gain at each split. Information gain can be understood as the difference between the impurity of a parent node and the sum of impurities of its child nodes. Thus, less impurity on child nodes, more information gain.

Random Forest is an ensemble of decision trees. Ensemble is a method that combines many models. The final prediction can be, for example, the average of them (regression) or the majority votes (classification). Random Forest consists in a group of decision trees built on samples of training dataset. More specifically, what happens is a sampling with replacement (bootstrap sampling, bagging ensemble method).

But why "random"? Because at each split, instead of choosing the feature that maximize de information gain among all features, will be considered only a random subset of the features. This brings some diversity to the model.

Random Forest is easy, robust and can collaborate with interpretability. Some of the advantages of Random Forest are the possibility of check feature importance.

In [ ]:
rf= RandomForestClassifier(random_state=42)

pg_rf={'n_estimators': [20,40,120],'max_depth': [5,10,15,30]} #n_estimators parameter= number of trees

gs_rf=GridSearchCV(estimator= rf,
               param_grid= pg_rf,
               scoring='f1',
               cv=2)

# nested cross validation combining grid search (inner loop) and k-fold cv (outter loop)
gs_rf_scores = cross_val_score(gs_rf, X=X_train, y=y_train, cv=5,scoring='f1', n_jobs=-1)

# fit, and fit with best estimator
gs_rf.fit(X_train, y_train)
gs_rf_best=gs_rf.best_estimator_
gs_rf.best_estimator_

In [ ]:
gs_rf_best.fit(X_train, y_train)
y_pred = gs_rf_best.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# using random forest here to get feature importances
importances= gs_rf_best.feature_importances_
feature_importances= pd.Series(importances, index=X_train.columns).sort_values(ascending=False)
sns.barplot(x=feature_importances[0:10], y=feature_importances.index[0:10], palette="rocket")
sns.despine()
plt.xlabel("Feature Importances")
plt.ylabel("Features")
plt.title("Most Important Features according to the Model with Random Forest")
plt.show()

In [ ]:
from plotnine import *
df2=df.groupby('tenure').agg(Churned=pd.NamedAgg('Churn', 'sum'),
                             Costumers=pd.NamedAgg('Churn', 'count')).reset_index()
df2['Churn_Rate']=round(df2['Churned']/df2['Costumers'], 2)


df2['Color']=np.where(df2['Churn_Rate']>0.4, 'blue', 'grey')

(
    ggplot(df2, aes(x='tenure',y='Churn_Rate', fill='Color'))+
    geom_bar(stat="identity")+
    labs(title="xxxxxxxxx", x= "tenure")+
    theme_classic()+
    theme(figure_size=(10, 4))+
    scale_fill_manual(values=['blue', 'grey'],guide=False)+
    annotate("text", x = 20, y = 0.45, color='blue',
             label = "xxxxxxxxxxxxxxxxxxxxxxxxxx")
).draw();